<a href="https://colab.research.google.com/github/tranthuc2000/CS114.K21/blob/master/DectectSarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CASE STYDY: Sarcasm Detection**

Nhóm:


1.   Trần Quốc Toàn : 18520384
1.   Nguyễn Tri Đức Anh : 18520463
2.   Trần Văn Thục : 18521474





## **Mô tả bài toán: phát hiện tin châm biếm**
Dataset được lấy từ 2 trang báo: 


*   HuffPost: chuyên viết tin chính thống.
*   The Onion: chuyên viết tin châm biếm

Dataset bao gồm 3 colums:
1.   headline : tiêu đề bài báo
2.   is_sacastic : là tin chính thống hay châm biếm 
3.   article_link : link bài báo


Cào dữ liệu dataset.csv bằng Data Scraper - Easy Web Scraping trên chrome extention 
Link file:https://github.com/tranthuc2000/CS114.K21/blob/master/datatest.csv

Link tiện ích:https://chrome.google.com/webstore/detail/data-scraper-easy-web-scr/nndknepjnldbdbepjfgmncbggmopgden

**Import thư viện để liên kết với drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


mình đang có gì trong thư mục hiện tại



In [ ]:
ls

drive/        Sarcasm_Headlines_Dataset.json
sample_data/  Sarcasm_Headlines_Dataset_v2.json


In [ ]:
#giải nén dataset
!unzip 'drive/My Drive/detectsarcasm/news-headlines-dataset-for-sarcasm-detection.zip'

Archive:  drive/My Drive/detectsarcasm/news-headlines-dataset-for-sarcasm-detection.zip
replace Sarcasm_Headlines_Dataset.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Sarcasm_Headlines_Dataset_v2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


**Đọc file rồi gán vào biến dataframe, lệnh lines=True để đọc từng dòng**

In [ ]:
import pandas as pd
df_1 = pd.read_json("Sarcasm_Headlines_Dataset.json",lines=True)

**Đọc file test**

In [ ]:
df_2 = pd.read_csv("drive/My Drive/detectsarcasm/datatest.csv",)
df_2

In [ ]:
#xem thông tin của df_1 và df_2
df_1.info()
print("                  ")
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_link  26709 non-null  object
 1   headline      26709 non-null  object
 2   is_sarcastic  26709 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 626.1+ KB
                  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1937 entries, 0 to 1936
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Headline      1937 non-null   object
 1   Is_sarcastic  1937 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 30.4+ KB


**Xóa bỏ cột article link trong dataset, vì chỉ dùng cột headline và is_sarcastic**


In [ ]:
df_1=df_1.drop(labels="article_link",axis=1)

tải về tệp tin giúp cho việc loại bỏ các thứ không quan trọng như dấu câu,...

Chạy vòng lập ép kiểu dữ liệu thành string để thao tác sau này tránh bị lỗi

In [ ]:
import numpy as np
import string 
from string import digits
from string import punctuation
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
for i in df_1['headline']:
  i = str(i)

for a in df_2['Headline']:
  a = str(a)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


**Xem kết quả sau khi chạy**

In [ ]:
df_1

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
26704,american politics in moral free-fall,0
26705,america's best 20 hikes,0
26706,reparations and obama,0
26707,israeli ban targeting boycott supporters raise...,0


# Feature engineering:
1. Tách bỏ các dấu câu

In [ ]:
hl_clean_1 = []
hl_clean_2 = []

for i in df_1['headline']:
    clean = i.translate(str.maketrans('', '', punctuation))
    #xóa kí hiệu đặc biệt như % & '
    clean = clean.translate(str.maketrans('', '', digits))
    #Xóa số
    hl_clean_1.append(clean)

for a in df_2['Headline']:
    clean = a.translate(str.maketrans('', '', punctuation))
    clean = clean.translate(str.maketrans('', '', digits))
    hl_clean_2.append(clean)


2. Các headline thành các từ và cụm từ

In [ ]:
hl_tokens_1 = []
hl_tokens_2 = []
for i in hl_clean_1:
  i = i.split()
  #tách thành list
  hl_tokens_1.append(i)
for i in hl_clean_2:
  i = i.split()
  hl_tokens_2.append(i)  


3. Xác định loại từ

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)
# Init Lemmatizer 1
lemmatizer = WordNetLemmatizer()

hl_lemmatized_1 = []
for tokens in hl_tokens_1:
    lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
    hl_lemmatized_1.append(lemm)

 # Init Lemmatizer 2 
hl_lemmatized_2 = []
for tokens in hl_tokens_2:
    lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
    hl_lemmatized_2.append(lemm)

Vecto hóa 1 văn bản, giữ lại tối đa 2000 từ(khớp với 2000 headline dùng dể test) để dễ xử lí dữ liệu

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import numpy as np
max_features = 2000
max_token = len (hl_lemmatized_1)
tokenizer = Tokenizer (num_words= max_features)
tokenizer.fit_on_texts(hl_lemmatized_1)
sequences = tokenizer.texts_to_sequences(hl_lemmatized_1)
X_train = pad_sequences(sequences, maxlen = max_token)
# biến X_train thành dạng vector có dộ dài các vecto(mỗi vecto là 1 headline) có độ dài bằng nhau

max_features = 2000
max_token = len (hl_lemmatized_1)
tokenizer = Tokenizer (num_words= max_features)
tokenizer.fit_on_texts(hl_lemmatized_2)
sequences = tokenizer.texts_to_sequences(hl_lemmatized_2)
X_test = pad_sequences(sequences, maxlen = max_token)
#padding giúp các chuỗi trong X_test có độ dài bằng nhau

Using TensorFlow backend.


In [ ]:
X_test

array([[   0,    0,    0, ...,  962,   11,  132],
       [   0,    0,    0, ...,  409,    7,  220],
       [   0,    0,    0, ...,   11,  147,  221],
       ...,
       [   0,    0,    0, ...,   79,   60,   38],
       [   0,    0,    0, ...,   29, 1441,    9],
       [   0,    0,    0, ...,    1,    5,    4]], dtype=int32)

In [ ]:
Y_train = df_1['is_sarcastic'].values
Y_train = np.vstack(Y_train)
print (X_train)

Y_test = df_2['Is_sarcastic'].values
Y_test = np.vstack(Y_test)
#biến Y_test thành 1 ma trận

[[   0    0    0 ... 1890    7 1985]
 [   0    0    0 ...  166    9  145]
 [   0    0    0 ...   41   14    1]
 ...
 [   0    0    0 ...    0    9   73]
 [   0    0    0 ...  686  715  425]
 [   0    0    0 ...  432    7    3]]


In [ ]:
len(X_test)

1937

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf  =  RandomForestClassifier(n_estimators=300)
#chọn model 
clf.fit(X_train, Y_train)
#train dữ liệu, đưa vào model để lựa chọn ra công thức phù hợp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [ ]:
le = LabelEncoder()
y_result=le.fit_transform(Y_test)
y_pred=clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


IN ra kết quả so sánh

In [ ]:
print("Result: ", (y_pred == y_result).tolist().count(True)/len(y_result))

Result:  0.610738255033557


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_result,y_pred))

              precision    recall  f1-score   support

           0       0.58      0.75      0.65       938
           1       0.67      0.48      0.56       999

    accuracy                           0.61      1937
   macro avg       0.62      0.62      0.61      1937
weighted avg       0.63      0.61      0.60      1937



Ta có:
58% dự đoán 0 là đúng, 75% loại là "0" được dự đoán đúng

67% dự đoán 1 là đúng, 48% loại là "1" được dự đoán đúng

f1-score là độ chính xác model được tính theo công thức:

f1=2 * ((precision*recall)/(precision+recall))

# Đối chiếu performance của model trên dataset đã cho và trên 2000 headine mới

Cho train và test với tập dữ liệu đã cho để so sánh với tập test trên 2000 headline mới

In [ ]:
from sklearn.model_selection import train_test_split
Y_train = df_1['is_sarcastic'].values
Y_train = np.vstack(Y_train)

X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train,test_size=0.3, random_state = 42)
print (X_train)

[[   0    0    0 ...  647    2   22]
 [   0    0    0 ...  309   75  571]
 [   0    0    0 ...  218    7  824]
 ...
 [   0    0    0 ...  105    9  363]
 [   0    0    0 ...    5    2 1127]
 [   0    0    0 ... 1663   22    3]]


In [ ]:
len(X_test)

8013

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf  =  RandomForestClassifier(n_estimators=300)
#chọn model 
clf.fit(X_train, Y_train)
#train dữ liệu



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [ ]:
le = LabelEncoder()
y_result=le.fit_transform(Y_test)
y_pred=clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


in ra kết quả và đối chiếu 

In [ ]:
print("Result: ", (y_pred == y_result).tolist().count(True)/len(y_result))

Result:  0.6558093098714589


Đối chiếu performance của model trên dataset đã cho và trên 2000 headine mới ta thấy có sự chênh lệch, 0.61 và 0.66. Performance khi chạy trên dataset cao hơn(0.66), còn khi chạy trên 2000 headline mới thì thấp hơn

Nhận xét: khi được test trên 2000 headline mới không nằm trong bộ dữ liệu, kết quả đưa ra thấp hơn so mới kết quả mà model đã train

## Sacarsm detection cho một headline bất kỳ được nhập vào

In [ ]:
str1 = str(input("Nhap chuoi: "))

Nhap chuoi: i am very happy hihi


Xử lí dữ liệu

In [ ]:
test =[]
clean = str1.translate(str.maketrans('', '', punctuation))
clean = clean.translate(str.maketrans('', '', digits))
test.append(clean)

testn=[]
for i in test:
    i = i.split()
    testn.append(i)

testn_1 = []
for i in testn:
    lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in i]
    testn_1.append(lemm)

max_features = 2000
max_token = len(hl_lemmatized_1)
tokenizer = Tokenizer (num_words= max_features)
tokenizer.fit_on_texts(testn_1)
sequences = tokenizer.texts_to_sequences(testn_1)
X_test_1 = pad_sequences(sequences, maxlen = max_token)

So sánh và đưa ra kết quả xem đó là tiêu đề châm biếm hay chính thống

In [ ]:
y_pred=clf.predict(X_test_1)
kq = y_pred[0]
if (kq != 0):
  print("Not sarcasm")
else:
  print("Sarcasm")

Sarcasm


**save model với pickle**


In [ ]:
import pickle 
pickle.dump(clf,open('/content/drive/My Drive/Colab Notebooks/pickle/pickle.docx', 'wb'))


## Nhận xét về bài toán:

Bài toán được đưa ra rất hay, có dataset thu thập dễ dàng. Tuy nhiên độ chính xác chưa được cao. Bài toán có thể mở rộng ra rất nhiều vấn đề trong thực tế, áp dụng được vào nhiều lĩnh vực trong tương lai, nhất là trong truyền thông khi có thể nhanh chóng phát ra các loại tin tức giả, xác minh tin tức chính thống